# Part 1: Evaluating RBF Kernels

In [1]:
# Imports all the needed built in libraries
import numpy as np
import numba
from numba import cuda
import math

In [2]:
# Define Constants
sigma = .1
npoints = 400
nsources = 50

# Constructs the Grid
plot_grid = np.mgrid[0:1:npoints * 1j, 0:1:npoints * 1j]

# Creats a 3 by 480000 matrix for targets
targets_xy = np.vstack((plot_grid[0].ravel(),
                        plot_grid[1].ravel(),
                        np.zeros(plot_grid[0].size))).T
targets_xz = np.vstack((plot_grid[0].ravel(),
                        np.zeros(plot_grid[0].size),
                        plot_grid[1].ravel())).T
targets_yz = np.vstack((np.zeros(plot_grid[0].size),
                       plot_grid[0].ravel(),
                       plot_grid[1].ravel())).T

targets = np.vstack((targets_xy, targets_xz, targets_yz))
print(targets)

# Creats a Random array of 3 by 50 of sources
rand = np.random.RandomState(0)
sources = rand.rand(nsources, 3)
print(sources)

# Copy the sources and targets to the compute device
device_targets = cuda.to_device(targets)

device_sources = cuda.to_device(sources)

[[0.         0.         0.        ]
 [0.         0.00250627 0.        ]
 [0.         0.00501253 0.        ]
 ...
 [0.         1.         0.99498747]
 [0.         1.         0.99749373]
 [0.         1.         1.        ]]
[[0.5488135  0.71518937 0.60276338]
 [0.54488318 0.4236548  0.64589411]
 [0.43758721 0.891773   0.96366276]
 [0.38344152 0.79172504 0.52889492]
 [0.56804456 0.92559664 0.07103606]
 [0.0871293  0.0202184  0.83261985]
 [0.77815675 0.87001215 0.97861834]
 [0.79915856 0.46147936 0.78052918]
 [0.11827443 0.63992102 0.14335329]
 [0.94466892 0.52184832 0.41466194]
 [0.26455561 0.77423369 0.45615033]
 [0.56843395 0.0187898  0.6176355 ]
 [0.61209572 0.616934   0.94374808]
 [0.6818203  0.3595079  0.43703195]
 [0.6976312  0.06022547 0.66676672]
 [0.67063787 0.21038256 0.1289263 ]
 [0.31542835 0.36371077 0.57019677]
 [0.43860151 0.98837384 0.10204481]
 [0.20887676 0.16130952 0.65310833]
 [0.2532916  0.46631077 0.24442559]
 [0.15896958 0.11037514 0.65632959]
 [0.13818295 0.1965823

The above cell creates a 3 by 480000 matrix for targets and a random array of 3 by 50 for sources. Sources and targets are then copied to the compute device using cuda.to_device().

In [3]:
# Define the size of the ThreadBlock
SX = 16
SY = 32
# Define cj for f(x)
weights = rand.rand(len(sources))

@cuda.jit
def rbf_evaluation_cuda(sources, targets, weights, IntArr):
    """Evaluate the RBF sum using cuda:
    Inputs:
        sources: Sources array
        targets: Targets array
        weights: Weights array
        IntArr: Empty array for the intermediate results"""
    
    local_result = cuda.shared.array((SX, SY), numba.float32)
    local_targets = cuda.shared.array((SX, 3), numba.float32)
    local_sources = cuda.shared.array((SY, 3), numba.float32)
    local_weights = cuda.shared.array(SY, numba.float32)
    
    # Local coordinates
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    # Global coordinates
    px, py = cuda.grid(2)
    
    if px >= targets.shape[0]:
        return
    
    # At first we are loading all the targets into the shared memory
    # We use only the first column of threads to do this.
    
    if ty == 0:
        for index in range(3):
            local_targets[tx, index] = targets[px, index]
    
    # We are now loading all the sources and weights.
    # We only require the first row of threads to do this.
    
    if tx == 0:
        for index in range(3):
            local_sources[ty, index] = sources[py, index]
        local_weights[ty] = weights[py]
        
    # Let us now sync all threads
    
    cuda.syncthreads()
    
    # Now compute the interactions
    
    squared_diff = numba.float32(0)
    
    for index in range(3):
        squared_diff += (local_targets[tx, index] - local_sources[ty, index])**2
    local_result[tx, ty] = math.exp(-squared_diff / (numba.float32(2) * numba.float32(sigma)**2)) * local_weights[ty]
    
    cuda.syncthreads()
    
    # Now sum up all the local results into an intermediate array
    
    block_y = cuda.blockIdx.y
    if ty == 0:
        res = numba.float32(0)
        for index in range(SY):
            res += local_result[tx, index]
        IntArr[px, block_y] = res  

The cell above defines the function for the rbf evaluation with cuda. The function starts by defining all the local variables using the threadblock size. Local coordinates and global coordinates are then defined and used to move the threads from a local to a global position and split the grid into threadblocks. In each block the columns are summed and used to construct a (m,p) grid known as the intermediate array.

In [4]:
# Defines the number of blocks in the horizontal and vertical
lblocks = (targets.shape[0] + SX - 1) // SX
pblocks = (sources.shape[0] + SY - 1) // SY

# creates empty array
IntArr = cuda.device_array((len(targets),pblocks), dtype=np.float32)

# Uses function
rbf_evaluation_cuda[(lblocks, pblocks), (SX, SY)](sources.astype('float32'), targets.astype('float32'), weights.astype('float32'), IntArr)

IntArr = IntArr.copy_to_host()
print(IntArr)

[[1.8632311e-03 3.0624286e-10]
 [1.9987950e-03 3.2998246e-10]
 [2.1429064e-03 3.5534681e-10]
 ...
 [8.0545660e-06 1.2174401e-08]
 [7.9701322e-06 1.0615391e-08]
 [7.8819048e-06 9.2506278e-09]]


The previous cell was used to define the number of blocks in the horizontal and vertical and test the function. I tested the intermediate result using the visualise function from the lecture notes to make sure my graphs matched the provided ones. 

In [5]:
#Summation Kernal

@cuda.jit
def sum_kernal(IntArr,results):
    """Evaluate the sum using cuda:
    Inputs:
        IntArr: Intermediate results
        results: Empty array fro list of sums"""
    # Local coordinates
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    # Global coordinates
    px, py = cuda.grid(2)
    
    block_y = cuda.blockIdx.y
    
    res = numba.float32(0)
    for i in range(block_y):
        res += IntArr[px,i]
    results[px] = res
    

The summation kernal works by summing the rows in the (m,p) grid and putting the values in a list of m length.

In [6]:
# The use of the summation kernal
results = cuda.device_array(len(targets), dtype=np.float32)

sum_kernal[(lblocks, pblocks), (SX, SY)](IntArr,results)
results = results.copy_to_host()
print(results)

[1.8632311e-03 1.9987950e-03 2.1429064e-03 ... 8.0545660e-06 7.9701322e-06
 7.8819048e-06]


## CPU Implementation

In [7]:
# CPU version of the rbf evaluation so the results arrays can be compared 
@numba.njit(parallel=True)
def rbf_evaluation(sources, targets, weights, result):
    """Evaluate the RBF sum."""
    
    n = len(sources)
    m = len(targets)
        
    result[:] = 0
    for index in numba.prange(m):
        result[index] = np.sum(np.exp(-np.sum(np.abs(targets[index] - sources)**2, axis=1) / (2 * sigma**2)) * weights)

In [8]:
result = np.zeros(len(targets), dtype=np.float32)
rbf_evaluation(sources, targets, weights, result)
print(result)

[1.8632307e-03 1.9987959e-03 2.1429050e-03 ... 8.0667323e-06 7.9807414e-06
 7.8911480e-06]


As can be seen from both lists my cuda implementation of rbf evaluation does work as every value is the same to 1 decimal place and both give the same visualisation.

In [9]:
%%timeit 
rbf_evaluation_cuda[(lblocks, pblocks), (SX, SY)](sources.astype('float32'), targets.astype('float32'), weights.astype('float32'), IntArr)
sum_kernal[(lblocks, pblocks), (SX, SY)](IntArr, results)

15.8 ms ± 224 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%timeit rbf_evaluation(sources, targets, weights, results)

241 ms ± 666 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


As can be seen from the displayed times the cuda implementation is more than 20 times faster. As the value of nsources increases the CPU implementation time increased at a higher rate than the GPU implementation.

# Part 2: Evaluating the discrete Laplacian on GPUs

In [11]:
# GPU implementation

@cuda.jit
def discretise_poisson_cuda(N, row_ind, col_ind, resultArr, f):
    """Generate the matrix and rhs associated with the discrete Poisson operator in cuda:
        Inputs:
        N: The dimension
        row_ind: Row indices
        col_ind: Column indices
        resultArr: Empty array of results
        f: Binary array
        """
    
    # Local coordinates
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    # Global coordinates
    px, py = cuda.grid(2)
    
    if ty == 0 and tx == 0:
        count = 0
        
    else:
        if tx == 0 or tx == N - 1 or ty == 0 or ty == N - 1:
            row_ind[count] = col_ind[count] = ty * N + tx
            resultArr[count] =  1
            f[ty * N + tx] = 0
            count += 1
        else:
            row_ind[count : count + 5] = ty * N + tx
            col_ind[count] = ty * N + tx
            col_ind[count + 1] = ty * N + tx + 1
            col_ind[count + 2] = ty * N + tx - 1
            col_ind[count + 3] = (ty + 1) * N + tx
            col_ind[count + 4] = (ty - 1) * N + tx
                                
            resultArr[count] = 4 * (N - 1)**2
            resultArr[count + 1 : count + 5] = - (N - 1)**2
            f[ty * N + tx] = 1
                
            count += 5         

The cell above defines the function for the discrete Laplacian with cuda. The function sets a counter at the first thread in the grid and proceeds to check if the next thread is at the boundary of the grid or not. If a thread is at the boundary of the grid its value in the results array will be assigned to 1 and 0 in the binary array f. If the thread is not at the boundary of the grid the values in the results array will be determined by $4(N - 1)^2$ and $-(N - 1)^2$ a 1 will be added to f.

In [12]:
# Defining required terms
N = 10

nelements = 5 * N**2 - 16 * N + 16

row_ind = cuda.device_array(nelements, dtype=np.float64)
col_ind = cuda.device_array(nelements, dtype=np.float64)
resultArr = cuda.device_array(nelements, dtype=np.float64)

f = cuda.device_array(N*N, dtype=np.float64)

# Using function
discretise_poisson_cuda[(16,16), (N,N)](N, row_ind, col_ind, resultArr, f)

In [13]:
print(f.copy_to_host())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


Obtained the correct array for the binary representation.

In [14]:
print(resultArr.copy_to_host())

[  1. -81. -81. -81. -81.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0. 

Was unable to correctly output the results array when compared to the CPU implementation below. I believe this is due to the count not being recorded correctly leading to the incorrect placement of terms. I was able to resolve by adding the for loops from the cpu implementation, but this did not seem to be the correct method as it did not utilise cudas ability to run loops alone.

## CPU Implementation

In [15]:
from scipy.sparse import coo_matrix

def discretise_poisson(N):
    """Generate the matrix and rhs associated with the discrete Poisson operator."""
    
    nelements = 5 * N**2 - 16 * N + 16
    
    row_ind = np.empty(nelements, dtype=np.float64)
    col_ind = np.empty(nelements, dtype=np.float64)
    data = np.empty(nelements, dtype=np.float64)
    
    f = np.empty(N * N, dtype=np.float64)
    
    count = 0
    for j in range(N):
        for i in range(N):
            if i == 0 or i == N - 1 or j == 0 or j == N - 1:
                row_ind[count] = col_ind[count] = j * N + i
                data[count] =  0
                f[j * N + i] = 0
                count += 1
                
            else:
                row_ind[count : count + 5] = j * N + i
                col_ind[count] = j * N + i
                col_ind[count + 1] = j * N + i + 1
                col_ind[count + 2] = j * N + i - 1
                col_ind[count + 3] = (j + 1) * N + i
                col_ind[count + 4] = (j - 1) * N + i
                                
                data[count] = 4 * (N - 1)**2
                data[count + 1 : count + 5] = - (N - 1)**2
                f[j * N + i] = 1
                
                count += 5
                                                
    return coo_matrix((data, (row_ind, col_ind)), shape=(N**2, N**2)).tocsr(), f

In [16]:
A,f = discretise_poisson(10)
print(f)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [17]:
print(A.data)

[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -81. -81. 324.
 -81. -81. -81. -81. 324. -81. -81. -81. -81. 324. -81. -81. -81. -81.
 324. -81. -81. -81. -81. 324. -81. -81. -81. -81. 324. -81. -81. -81.
 -81. 324. -81. -81. -81. -81. 324. -81. -81.   0.   0. -81. -81. 324.
 -81. -81. -81. -81. 324. -81. -81. -81. -81. 324. -81. -81. -81. -81.
 324. -81. -81. -81. -81. 324. -81. -81. -81. -81. 324. -81. -81. -81.
 -81. 324. -81. -81. -81. -81. 324. -81. -81.   0.   0. -81. -81. 324.
 -81. -81. -81. -81. 324. -81. -81. -81. -81. 324. -81. -81. -81. -81.
 324. -81. -81. -81. -81. 324. -81. -81. -81. -81. 324. -81. -81. -81.
 -81. 324. -81. -81. -81. -81. 324. -81. -81.   0.   0. -81. -81. 324.
 -81. -81. -81. -81. 324. -81. -81. -81. -81. 324. -81. -81. -81. -81.
 324. -81. -81. -81. -81. 324. -81. -81. -81. -81. 324. -81. -81. -81.
 -81. 324. -81. -81. -81. -81. 324. -81. -81.   0.   0. -81. -81. 324.
 -81. -81. -81. -81. 324. -81. -81. -81. -81. 324. -81. -81. -81. -81.
 324. 

In [18]:
%timeit discretise_poisson_cuda[(16,16), (N,N)](N, row_ind, col_ind, resultArr, f)

620 µs ± 11 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [19]:
%timeit discretise_poisson(N)

300 µs ± 3.56 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


As can be seen from above the time it took the cuda implementation was longer than the CPU implementation which is not the expected result. I believe this is due to the error in my cuda implementation. If this error were to be resolved I am confident that the cuda implementation will be faster.

# Part 2: Solving modifed Helmholtz problems with CG on the GPU

In [ ]:
import numpy as np
from scipy.sparse.linalg import LinearOperator